In [29]:
import numpy as np
import pandas as pd


class MyTreeClf():

    class Tree():
        pass 
    
    def __init__(self, max_depth, min_samples_split, max_leafs) -> None:
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.leafs_cnt = 0

        self.lst = []
        self.np_lst = np.array([[]])
        self.dct = dict()

    def __str__(self) -> str:
        return f"MyTreeClf class: max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}"

    def __repr__(self) -> str:
        return f"MyTreeClf class: max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}"
    
    def get_best_split(self, X, y):
        main_dict = {}

        def S(p_0, p_1):
            if p_0 != 0:
                part_p_0 = p_0 * np.log2(p_0)
            else:
                part_p_0 = 0
            if p_1 != 0:
                part_p_1 = p_1 * np.log2(p_1)
            else:
                part_p_1 = 0
            return -np.sum(part_p_0 + part_p_1)

        for col in X.columns:
            one_col = np.sort(X[col].unique())
            tresholds = one_col[:-1] + np.diff(one_col) / 2

            proba_zero = np.sum(y==0) / len(y)
            proba_one = np.sum(y==1) / len(y)
            S_0 = S(proba_zero, proba_one)

            for treshold in tresholds:

                y_right = y[X[col] <= treshold]
                proba_zero_left = np.sum(y_right==0) / len(y_right)
                proba_one_left  = np.sum(y_right==1) / len(y_right)
                S_left = S(proba_zero_left, proba_one_left)* len(y_right)/len(y)

                y_left = y[X[col] > treshold]
                proba_zero_right = np.sum(y_left==0) / len(y_left)
                proba_one_right  = np.sum(y_left==1) / len(y_left)
                S_right = S(proba_zero_right, proba_one_right) * len(y_left)/len(y)

                IG = S_0 - (S_left + S_right)
                if IG in main_dict.keys():
                    pass
                else:
                    main_dict[IG] = (treshold, col)
        best_split = max(main_dict.keys())
        return main_dict[best_split][1], main_dict[best_split][0], best_split

    def node_or_list(self, y, current_depth):
        if len(y) == 1 or \
        y.nunique() == 1 or \
        self.min_samples_split > len(y) or\
        self.max_leafs <= self.leafs_cnt or\
        current_depth >= self.max_depth:
            return True

    def fit(self, X, y, current_depth=0, side=None):

        if self.node_or_list(y, current_depth):
            self.lst.append([current_depth, '    '*current_depth + f'{str(side)} ={np.mean(y)}'])
            return self
        
        else:
            if self.leafs_cnt == 0:
                self.leafs_cnt += 2
            else:
                self.leafs_cnt += 1
                
            feature, thresh, ig = self.get_best_split(X, y)
            X_left = X[X[feature] < thresh]
            y_left = y[X[feature] < thresh]

            X_right = X[X[feature] >= thresh]
            y_right = y[X[feature] >= thresh]
            
            self.lst.append([current_depth, '    '*current_depth + f'{feature} > {thresh}'])

            # first recur
            self.fit(X_left, y_left, current_depth+1, side='leaf_left')

            # second recur
            # self.lst.append(str(current_depth) +' ' + '    '*current_depth + f'{feature} <= {thresh}')
            self.fit(X_right, y_right, current_depth+1, side='leaf_right')

    def print_tree(self):
        return self.lst
                    
                    
    def predict(self, X):
        '''
        На вход принимать матрицу фичей в виде датафрейма пандаса.
        Для каждой строки из датафрейма:
        Пройтись по узлам дерева до одного из листьев.
        Записать вероятность для первого класса.
        Вернуть список предсказаний – столько, сколько строк в датафрейме.
        '''
        X
        